<a href="https://colab.research.google.com/github/santiagonajera/Optimizacion-Global-Inventarios/blob/main/ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Cargar los datos desde el archivo Excel
url = "https://github.com/santiagonajera/Optimizacion-Global-Inventarios/raw/refs/heads/main/Clases-Keisser2026.xlsx"
df_forecast = pd.read_excel(url, sheet_name='Forecast')
df_precios = pd.read_excel(url, sheet_name='Precios-Costos')

# Limpiar y preparar datos de forecast
df_forecast.columns = ['ITEM'] + [f'mes_{i+1}' for i in range(12)]
df_forecast['ITEM'] = df_forecast['ITEM'].str.strip()
df_forecast['venta_total_2026'] = df_forecast.iloc[:, 1:13].sum(axis=1)

# Limpiar y preparar datos de precios
df_precios['ITEM'] = df_precios['ITEM'].str.strip()
df_precios['Precio'] = df_precios['Precio'].replace('[\$,]', '', regex=True).astype(float)

# Combinar ambos datasets
df_merged = pd.merge(df_forecast[['ITEM', 'venta_total_2026']],
                     df_precios[['ITEM', 'Precio']],
                     on='ITEM',
                     how='inner')

# Calcular venta en dólares
df_merged['venta_dolares'] = df_merged['venta_total_2026'] * df_merged['Precio']

# Calcular contribución y clasificación ABC
df_merged = df_merged.sort_values('venta_dolares', ascending=False)
df_merged['contribucion'] = df_merged['venta_dolares'] / df_merged['venta_dolares'].sum() * 100
df_merged['contribucion_acumulada'] = df_merged['contribucion'].cumsum()

# Asignar clasificación ABC
conditions = [
    df_merged['contribucion_acumulada'] <= 60,
    (df_merged['contribucion_acumulada'] > 60) & (df_merged['contribucion_acumulada'] <= 80),
    df_merged['contribucion_acumulada'] > 80
]
choices = ['A', 'B', 'C']
df_merged['clasificacion_ABC'] = np.select(conditions, choices, default='C')

# Resultados finales
resultado_final = df_merged[['ITEM', 'venta_dolares', 'contribucion',
                             'contribucion_acumulada', 'clasificacion_ABC']]

print("Resumen de clasificación ABC:")
print(resultado_final)

print("\nDistribución de categorías:")
print(resultado_final['clasificacion_ABC'].value_counts())


Resumen de clasificación ABC:
         ITEM  venta_dolares  contribucion  contribucion_acumulada  \
11    ITEM 12     3190090.80      0.790204                0.790204   
1      ITEM 2     2881611.78      0.713792                1.503996   
0      ITEM 1     2751451.56      0.681551                2.185547   
6      ITEM 7     2664927.45      0.660118                2.845665   
44    ITEM 45     2455450.98      0.608230                3.453895   
..        ...            ...           ...                     ...   
88    ITEM 89      720814.72      0.178550               99.357351   
146  ITEM 147      714271.70      0.176929               99.534280   
116  ITEM 117      701536.32      0.173775               99.708055   
114  ITEM 115      697497.90      0.172774               99.880829   
27    ITEM 28      481097.96      0.119171              100.000000   

    clasificacion_ABC  
11                  A  
1                   A  
0                   A  
6                   A  
44       